In [1]:
import pywt
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import math
import os


%matplotlib qt5

In [2]:
pywt.wavelist()

['bior1.1',
 'bior1.3',
 'bior1.5',
 'bior2.2',
 'bior2.4',
 'bior2.6',
 'bior2.8',
 'bior3.1',
 'bior3.3',
 'bior3.5',
 'bior3.7',
 'bior3.9',
 'bior4.4',
 'bior5.5',
 'bior6.8',
 'cgau1',
 'cgau2',
 'cgau3',
 'cgau4',
 'cgau5',
 'cgau6',
 'cgau7',
 'cgau8',
 'cmor',
 'coif1',
 'coif2',
 'coif3',
 'coif4',
 'coif5',
 'coif6',
 'coif7',
 'coif8',
 'coif9',
 'coif10',
 'coif11',
 'coif12',
 'coif13',
 'coif14',
 'coif15',
 'coif16',
 'coif17',
 'db1',
 'db2',
 'db3',
 'db4',
 'db5',
 'db6',
 'db7',
 'db8',
 'db9',
 'db10',
 'db11',
 'db12',
 'db13',
 'db14',
 'db15',
 'db16',
 'db17',
 'db18',
 'db19',
 'db20',
 'db21',
 'db22',
 'db23',
 'db24',
 'db25',
 'db26',
 'db27',
 'db28',
 'db29',
 'db30',
 'db31',
 'db32',
 'db33',
 'db34',
 'db35',
 'db36',
 'db37',
 'db38',
 'dmey',
 'fbsp',
 'gaus1',
 'gaus2',
 'gaus3',
 'gaus4',
 'gaus5',
 'gaus6',
 'gaus7',
 'gaus8',
 'haar',
 'mexh',
 'morl',
 'rbio1.1',
 'rbio1.3',
 'rbio1.5',
 'rbio2.2',
 'rbio2.4',
 'rbio2.6',
 'rbio2.8',
 'rbio3.1',

In [4]:
path = r'C:\Users\Yuan\Desktop\316L-1.5-z1-AE-20180921_wf'
os.chdir(path)
file = os.listdir(path)
# for i in [0,12,19,23]:
for i in [1,8,10,20]:
    with open(file[i], 'r') as f:
        f.readline()
        data= np.array([float(i.strip()) for i in f.readlines()])

    plot_signal_decomp(data, 'db8', "DWT: Sample - db8")

In [3]:
def plot_signal_decomp(data, w, title):
    """Decompose and plot a signal S.
    S = An + Dn + Dn-1 + ... + D1
    """
    w = pywt.Wavelet(w) #选取小波函数
    a = data
    ca = [] #近似分量
    cd = [] #细节分量
    for i in range(5):
        (a, d) = pywt.dwt(a, w, pywt.Modes.smooth) #进行5阶离散小波变换
        ca.append(a)
        cd.append(d)

    rec_a = []
    rec_d = []

    for i, coeff in enumerate(ca):
        """
        waverec()返回list，[cAn, cDn, cDn-1, …, cD2, cD1]，
        n为分解阶次，cAn是逼近系数数组，后面的依次是细节系数数组
        [coeff, None] + [None] * i 是重构低频
        """
        coeff_list = [coeff, None] + [None] * i
        rec_a.append(pywt.waverec(coeff_list, w)) #重构

    for i, coeff in enumerate(cd):
        """
        [coeff, None] + [None] * i 是重构对角线高频
        """
        coeff_list = [None, coeff] + [None] * i
        rec_d.append(pywt.waverec(coeff_list, w))

    maxlev = pywt.dwt_max_level(data.shape[0], w.dec_len)
    threshold = 0.04
    coeffs = pywt.wavedec(data, 'db8', level=maxlev)
    for i in range(1, len(coeffs)):
        coeffs[i] = pywt.threshold(coeffs[i], threshold * max(coeffs[i]))
    datarec = pywt.waverec(coeffs, 'db8')
        
    fig = plt.figure()
    title = ['Raw signal', 'De-noised signal using wavelet techniques']
    y = [data, datarec]
    for idx, [i, j] in enumerate(zip(title, y)):
        ax_main = fig.add_subplot(len(rec_a) + 2, 1, idx + 1)
        ax_main.set_title(i)
        ax_main.plot(j)
        ax_main.set_xlim(0, len(data) - 1)

    for i, y in enumerate(rec_a):
        ax = fig.add_subplot(len(rec_a) + 2, 2, 5 + i * 2)
        ax.plot(y, 'r')
        ax.set_xlim(0, len(y) - 1)
        ax.set_ylabel("A%d" % (i + 1))

    for i, y in enumerate(rec_d):
        ax = fig.add_subplot(len(rec_d) + 2, 2, 6 + i * 2)
        ax.plot(y, 'g')
        ax.set_xlim(0, len(y) - 1)
        ax.set_ylabel("D%d" % (i + 1))
    plt.tight_layout()